# 07 Semantic Kernel | 01 Plugin

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

This notebooks highlights how Semantic Kernel simplifies the process of storing and retrieving vector data. 

## Step 1: Create Semantic Kernel

MS Semantic Kernel is a tool or library developed by Microsoft to abstract away communication with LLMs and other services like vector DBs. The nuget package Microsoft.SemanticKernel provides developers with access to this powerful tool, allowing them to integrate advanced language processing capabilities into their applications effortlessly.


In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"
#r "nuget: System.Linq.Async, 6.0.1"
#r "nuget: DotNetEnv, 2.5.0"

using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Kernel = Microsoft.SemanticKernel.Kernel;
using DotNetEnv;

//configuration file is created during environment creation
static string _configurationFile = @"../Configuration/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI1106_APIKEY") ?? "WS_AOAI1106_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI1106_ENDPOINT") ?? "WS_AOAI1106_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_AOAI1106_DEPLOYMENTNAME") ?? "WS_AOAI1106_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_EMBEDDING_DEPLOYMENTNAME not found";


var kernel = Kernel
    .CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        chatCompletionDeploymentName,
        oAiEndpoint,
        oAiApiKey,
        serviceId: chatCompletionDeploymentName 
    )
    .Build();
Console.WriteLine($"Semantic Kernel created...");


Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha System.Linq.Async, 6.0.1

Semantic Kernel created...


## Step 2: Create Memory Store

Semantic Kernel offers an abstraction of creating embeddings and storing/querying them in connected vector databases. 

The `MemoryBuilder()` class offers a function `WithMemoryStore()` which allows abstracting the used vector database. In the code cell a volatile memory store which can be used for development purposes is used.


In [2]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Memory functionality is experimental
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0050, SKEXP0011, SKEXP0003, SKEXP0052

MemoryBuilder memoryBuilder = new MemoryBuilder();

memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
    embeddingDeploymentName,
    oAiEndpoint, 
    oAiApiKey,
    embeddingDeploymentName);

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());

ISemanticTextMemory semanticTextMemory = memoryBuilder.Build();

Console.WriteLine($"SemanticTextMemory created...");

SemanticTextMemory created...


## Step 3: Add Facts to Memory Store
The code cell adds specific information or knowledge to the previously created memory. Adding information to the memory means: 

- create embeddings out of the information
- store the embeddings together with the information in the connected memory store (e.g.: Vector DB)

In [5]:
string collection = "aboutMe";

//define, embedd & store artificial facts
string factName = "Company Music";
string factDescription = @"Company Musik is one of the world's leading record labels. 
                            It has signed famous singers and is very profitable! 
                            The flagship of Contoso Music is a group that performs under the name 'Contoso Only'!";
await semanticTextMemory.SaveInformationAsync(collection, id: factName, text: factDescription);
Console.WriteLine($"Information about: {factName} saved in memory...");

await Task.Delay(TimeSpan.FromSeconds(5)); //avoid throttling
factName = "Company Maritim";
factDescription = @"Company Heavy Industry Maritime products. 
                    The current bestseller is the container transporter 'Contoso XL Heavy 2000'.";
await semanticTextMemory.SaveInformationAsync(collection, id: factName, text: factDescription);
Console.WriteLine($"Information about: {factName} saved in memory...");

await Task.Delay(TimeSpan.FromSeconds(5)); //avoid throttling
factName = "Company Agriculture";
factDescription = @"Company Agriculture is a German start-up that focuses on the production of milk and grain. 
                    Since this is a start-up, no further information is available!";
await semanticTextMemory.SaveInformationAsync(collection, id: factName, text: factDescription);
Console.WriteLine($"Information about: {factName} saved in memory...");

Information about: Company Music saved in memory...
Information about: Company Maritim saved in memory...
Information about: Company Agriculture saved in memory...


## Step 4: Query Memory Store

The code cell uses the `SearchAsync()` function to query the memory store. 

In [6]:
string query = "Who produces Container Ships?";

var response = await semanticTextMemory.SearchAsync(collection, query).FirstOrDefaultAsync();

Console.WriteLine($"Response: {response?.Metadata.Text} - Relevance: {response?.Relevance}");

Response: Company Heavy Industry Maritime products. 
                    The current bestseller is the container transporter 'Contoso XL Heavy 2000'. - Relevance: 0.8797731399536133
